In [32]:
# Download in terminal: 
# pip install fix_yahoo_finance --upgrade --no-cache-dir
import pandas as pd
import matplotlib.pyplot as plt
from pandas_datareader import data as pdr
import fix_yahoo_finance as yf 
# Download zipline if not already installed
# pip install zipline
import zipfile

In [33]:
# Stocks that we are interested in 
stocks  = "AAPL AMZN BA FB GOOG MA MSFT NVDA UNH V"

# Create a table with dates and close price
data = yf.download(tickers=stocks,
                  start = '2013-05-18',
                  end = '2018-12-31',
                  interval = '1d',
                  group_by='column')
data
StockClose = data['Close']
StockClose

[*********************100%***********************]  10 of 10 downloaded


,AAPL,AMZN,BA,FB,GOOG,MA,MSFT,NVDA,UNH,V
Date,,,,,,,,,,
2013-05-20,63.28,267.63,98.72,25.76,451.33,58.33,35.08,14.84,62.56,45.40
2013-05-21,62.81,268.86,98.75,25.66,450.55,58.49,34.85,14.92,62.87,45.39
2013-05-22,63.05,262.96,97.93,25.16,441.84,57.67,34.61,14.40,62.28,45.03
2013-05-23,63.16,261.80,99.75,25.06,438.54,56.97,34.15,14.63,62.36,44.53
2013-05-24,63.59,261.74,100.00,24.31,433.84,57.15,34.27,14.54,62.06,45.11
2013-05-28,63.06,267.29,100.11,24.10,437.79,57.57,35.02,14.53,63.34,45.01
2013-05-29,63.56,265.53,99.09,23.32,431.35,57.30,34.88,14.44,63.35,44.53
2013-05-30,64.51,266.83,100.54,24.55,432.57,57.62,35.03,14.63,64.66,45.26
2013-05-31,64.25,269.20,99.02,24.35,432.80,57.03,34.90,14.47,62.63,44.53
